# Defining Flow Stress from Interpolated Data - Using Lambda Functions

In this notebook it is shown, how to define a flow stress function from interpolated data using the interpolation functions from `scipy`.
The concept can be transferred to any other material data as well.

This notebook is essentially the same as `Interpolating_Flow_Stress.ipynb`, with one main difference: we will use lambda functions as hook values instead of real hook implementations. This is appropriate for usage in input files and for quick testing purposes, plugins shall and can not use this approach.

First, we need to import some packages.

In [1]:
import pyroll.basic as pr # PyRolL itself

import pandas as pd # pandas for data import
import numpy as np # numpy for numeric calculations
import scipy.interpolate as interpolate # scipy.interpolate for interpolations

We import the data to interpolate from a CSV file using pandas. The result is a dataframe containing all data from the file. Note, that the column names are derived from the first row of the file.

In [2]:
flow_stress_data = pd.read_csv("Interpolating_Flow_Stress_Data.csv")
flow_stress_data

,phi,kf
0,0.000000,100.000000
1,0.020202,106.203666
2,0.040404,107.637609
3,0.060606,108.625511
4,0.080808,109.403000
...,...,...
95,1.919192,124.320109
96,1.939394,124.396628
97,1.959596,124.472591
98,1.979798,124.548008


We can interpolate the data using scipy to get an interpolation function, which we can evaluate as any other function.

In [3]:
flow_stress_function = interpolate.interp1d(flow_stress_data.phi, flow_stress_data.kf)


Here comes the difference, we do not define hook implementations, but give the interpolation function directly to the in profile.
The `flow_stress` hook is the main hook for calculation flow stress, it returns the flow stress depending on the current state of the profile.
Another hook regarding flow stress is `flow_stress_function`. It returns a function of the classic flow stress parameters strain, strain rate and temperature to enable plugins to calculate their own variations of flow stress.
Here it needs a nested lambda function to be defined.

In [4]:
in_profile = pr.Profile.square(
    side=45e-3,
    corner_radius=3e-3,
    temperature=1100 + 273.15,
    strain=0,
    density=7.5e3,
    thermal_capacity=690,
    flow_stress=lambda self: flow_stress_function(self.strain),
    flow_stress_function=lambda self: lambda strain, strain_rate, temperature: flow_stress_function(strain)
)
in_profile

SquareProfile(
    classifiers={'diamond', 'square'},
    corner_radius=0.003,
    cross_section=Polygon(
        area=0.0020172289364149133,
        height=0.061154328932550704,
        perimeter=0.17484198694172845,
        width=0.061154328932550704,
        ),
    density=7500.0,
    diagonal=0.06363961030678927,
    flow_stress=<function __main__.<lambda>(self)>,
    flow_stress_function=<function __main__.<lambda>(self)>,
    side=0.045,
    strain=0,
    t=0,
    temperature=1373.15,
    thermal_capacity=690,
    )

And the groove, the working roll and the roll pass...

In [5]:
groove1 = pr.DiamondGroove(
    usable_width=76.55e-3,
    tip_depth=22.1e-3,
    r1=12e-3,
    r2=8e-3,
)
groove1

DiamondGroove(
    alpha1=0.5236363668157296,
    alpha2=0.5236363668157296,
    classifiers={'diamond', 'generic_elongation'},
    contour_line=LineString(
        depth=0.020862195195594648,
        length=0.12428942750252188,
        width=0.11360126410252692,
        ),
    depth=0.020862195195594648,
    flank_angle=0.5236363668157296,
    ground_width=0.004287509401684611,
    r1=0.012,
    r2=0.008,
    tip_angle=None,
    tip_depth=0.0221,
    usable_width=0.07655,
    )

In [6]:
roll1 = pr.Roll(
    groove=groove1,
    nominal_radius=328e-3,
)
roll1

Roll(
    groove=DiamondGroove(
        alpha1=0.5236363668157296,
        alpha2=0.5236363668157296,
        classifiers={'diamond', 'generic_elongation'},
        contour_line=LineString(
            depth=0.020862195195594648,
            length=0.12428942750252188,
            width=0.11360126410252692,
            ),
        depth=0.020862195195594648,
        flank_angle=0.5236363668157296,
        ground_width=0.004287509401684611,
        r1=0.012,
        r2=0.008,
        tip_angle=None,
        tip_depth=0.0221,
        usable_width=0.07655,
        ),
    nominal_radius=0.328,
    )

In [7]:
pass1 = pr.RollPass(
    roll=roll1,
    gap=3e-3,
    velocity=1,
)
pass1

RollPass(
    disk_elements=[],
    gap=0.003,
    in_profile=None,
    label='',
    out_profile=None,
    roll=Roll(
        groove=DiamondGroove(
            alpha1=0.5236363668157296,
            alpha2=0.5236363668157296,
            classifiers={'diamond', 'generic_elongation'},
            contour_line=LineString(
                depth=0.020862195195594648,
                length=0.12428942750252188,
                width=0.11360126410252692,
                ),
            depth=0.020862195195594648,
            flank_angle=0.5236363668157296,
            ground_width=0.004287509401684611,
            r1=0.012,
            r2=0.008,
            tip_angle=None,
            tip_depth=0.0221,
            usable_width=0.07655,
            ),
        nominal_radius=0.328,
        ),
    velocity=1,
    )

Lets run the solution procedure of the roll pass.
We see no errors claiming that a flow stress value is missing.

In [8]:
pass1.solve(in_profile)

Profile(
    classifiers={'diamond', 'generic_elongation'},
    cross_section=Polygon(
        area=0.0018470652317200606,
        height=0.0447243903911893,
        perimeter=0.1684260697843319,
        width=0.06560167360406856,
        ),
    cross_section_error=-0.0360789946906479,
    cross_section_filling_ratio=0.9639210053093521,
    density=7500.0,
    filling_error=-0.143021899359,
    filling_ratio=0.856978100641,
    flow_stress=<function __main__.<lambda>(self)>,
    flow_stress_function=<function __main__.<lambda>(self)>,
    length=0.0,
    strain=0.19349552465258638,
    t=0.07056039637464354,
    temperature=1369.1993358613693,
    thermal_capacity=690,
    )

Lets check, if our new hooks are really used. We check that the flow stress value stored in the out profile of the pass is numerically equal to a manually calculated value using the respective strain.
If everything went fine, we shall se no output of this statement, otherwise an error message.

In [9]:
assert np.isclose(pass1.out_profile.flow_stress, flow_stress_function(pass1.out_profile.strain))